# AutoML end-to-end with OpenFE and AutoGluon
This example notebook uses [OpenFE](https://openfe-document.readthedocs.io/en/latest/) and the [Boston Housing Dataset](https://www.kaggle.com/code/prasadperera/the-boston-housing-dataset) to create new features that are saved into the Feature Store and can be used for Machine Learning.

# UNSUPPORTED BY SNOWFLAKE - CUSTOMER SUPPORTED ONLY

# Copyright (c) 2025 Snowflake Inc. All rights reserved.

In [ ]:
# Install the OpenFE package
!pip install openfe --quiet

In [ ]:
# input data for feature engineering
table_name = 'DEMO_BOSTON_HOUSING'
# feature to be predicted
target_feature = 'medv'
# unique / key column name
feature_store_join_key = 'ID'

In [ ]:
# retrieve the notebook name as we will use it to name the feature store and deployment
notebook_name = os.environ.get('OBJECT_NAME', 'NOTEBOOK')
notebook_name = notebook_name.replace(' ','_')
print("Name that will be used to register features and other artifacts: "+notebook_name)

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
# import numpy
import numpy as np

# used for feature engineering
from openfe import OpenFE, transform, tree_to_formula

# used to creat train and test datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#Snowflake feature store
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode

# helper to set entry details based on Notebook
import os
import datetime

# helper for extracting new features
import re

In [ ]:
# get data from Snowflake. This is a public dataset
data = session.table(table_name).to_pandas()
data.head()

## Dataset Details
Each record in the database describes a Boston suburb or town. The data was drawn from the Boston Standard Metropolitan Statistical Area (SMSA) in 1970.

| **Name**    | **Description**                                                           |
|---------|-----------------------------------------------------------------------|
| CRIM    | per capita crime rate by town                                         |
| ZN      | proportion of residential land zoned for lots over 25000 sq.ft.       |
| INDUS   | proportion of non-retail business acres per town                      |
| CHAS    | Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) |
| NOX     | nitric oxides concentration (parts per 10 million)                    |
| RM      | average number of rooms per dwelling                                  |
| AGE     | proportion of owner-occupied units built prior to 1940                |
| DIS     | weighted distances to five Boston employment centres                  |
| RAD     | index of accessibility to radial highways                             |
| TAX     | full-value property-tax rate per 10000usd                             |
| PTRATIO | pupil-teacher ratio by town                                           |
| LSTAT   | % lower status of the population                                      |
| ID      | column used for Feature Store joins - represents suburb               |

In [ ]:
# identify the target the we will be predicting and remove it from the data used from training (inputs)
target = data.pop(target_feature)
inputs = data
join_key = data.pop(feature_store_join_key)

In [ ]:
# create a train and test dataset

x_train, x_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2, random_state=42)
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

In [ ]:
# create a pipeline to transform the data this will result in set of new features
ofe = OpenFE()

features = ofe.fit(data=x_train, label=y_train)

In [ ]:
# add the new features to the training and test datasets
train_f, test_f = transform(x_train, x_test, features, n_jobs=1) 
train_f[target_feature] = target
train_f[feature_store_join_key] = join_key

In [ ]:
# check the new features
train_f.head()

In [ ]:
# Create the sql for feature store. Groupby and Combine operations are ignored

# the first generated feature default name is autoFE_f_0
i = train_f.columns.get_loc('autoFE_f_0')-1

#feature_view_sql = 'select "crim", "zn", "indus", "chas", "nox", "rm", "age", "dis", "rad", "tax", "ptratio", "lstat", "ID"'
feature_view_sql = 'select *'
feature_dict = dict()

for feature in ofe.new_features_list:
    i = i +1
    #print(str(train_f.columns[i])+" "+str(tree_to_formula(feature)))
    new_feature_name = train_f.columns[i]
    new_feature_transform = tree_to_formula(feature)

    if new_feature_transform.startswith('Combine'):
        continue

    if new_feature_transform.startswith('max('):
        colnames = re.sub(r'([a-zA-Z]+)', r'"\1"', new_feature_transform)
        feature_view_sql = feature_view_sql +", array_max(["+colnames[6:].replace(")","]")+") as "+new_feature_name
        feature_dict[new_feature_name] = 'defined as array_max(['+colnames[6:].replace(")","]")+')'
        continue

    if not new_feature_transform.startswith('Group'):
       colnames = re.sub(r'([a-zA-Z]+)', r'"\1"', new_feature_transform)
       # fix up function name
       if colnames.startswith('"'):
            colnames = colnames[1:]
            colnames = colnames.replace('"','',1)

       feature_view_sql = str(feature_view_sql) +', '+str(colnames)+' as '+str(new_feature_name)
       feature_dict[new_feature_name] = 'defined as '+colnames

feature_view_sql = feature_view_sql + ' from '+table_name
print("Feature Store SQL:")
print(feature_view_sql)

## Use FeatureStore to save the features that OpenFE created 

In [ ]:
# connect to the feature store in the current database and schema
# if the feature store does not exist, it will be created

fs = FeatureStore(
    session=session, 
    database=session.get_current_database(), 
    name=session.get_current_schema(), 
    default_warehouse=session.get_current_warehouse(),
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

In [ ]:
# register a new entity if it does not exist

try:
    #retrieve existing entity
    entity = fs.get_entity(notebook_name) 
    print('Retrieved existing entity')
except:
    #define new entity
    entity = Entity(
        name = notebook_name,
        join_keys = [feature_store_join_key],
        desc = "OpenFE created features from Notebook "+notebook_name)
    #register
    fs.register_entity(entity)
    print("Registered new entity")

In [ ]:
# select the feature view that was created with the features from OpenFE

feature_df = session.sql(feature_view_sql)

feature_df.explain()

In [ ]:
# register the feature view created with OpenFE in the feature store
# call it version 1 and overwrite if it already exists

feature_fv = FeatureView(
    name=notebook_name+"_VIEW",
    entities=[entity],
    feature_df=feature_df,
    desc="OpenFE generated feature view from notebook "+notebook_name,
).attach_feature_desc(feature_dict)

registered_fv_v1 = fs.register_feature_view(feature_fv, version="1", overwrite=True)

In [ ]:
# show the feature view that was created

fs.list_feature_views()

In [ ]:
# get the entity based on the join key from the table that was defined earlier

entity_df = session.sql("select "+feature_store_join_key+" from "+table_name)


In [ ]:
# create a training set based on the registered feature view

training_df = fs.generate_training_set(
    spine_df=entity_df,
    features=[registered_fv_v1]
).to_pandas()

In [ ]:
# uppercase the columns names, it will be easier later.

training_df.columns = training_df.columns.str.upper()
training_df

In [ ]:
# write the training data from feature store to a table to training.
# we keep a copy for compliance and reproducabilty

timestamp = datetime.datetime.now().strftime("%Y_%m_%d")
training_table = table_name+"_TRAINING_"+timestamp

print("Writing table "+training_table+" use this table for model training.")
session.write_pandas(training_df, table_name=training_table, auto_create_table=True)

Now use one of the training notebooks to train a model using the above training table.